In [19]:

import os as os
import collections as col

import numpy as np
import pandas as pd

fhgfs_path = '/TL/deep/fhgfs/projects/pebert/thesis/projects/statediff'
quant_path = os.path.join(fhgfs_path, 'salmon', 'deep', 'quant')

merger = col.defaultdict(list)

outfile = os.path.join(fhgfs_path, 'salmon', 'deep', 'agg_gene_tpm.h5')

if not os.path.isfile(outfile):  
    print('Aggregating gene expression estimates...')

    for root, dirs, files in os.walk(quant_path):
        genefile = [f for f in files if f.endswith('.genes.sf')]
        for gf in genefile:
            sample = os.path.split(root)[-1][3:12]
            fpath = os.path.join(root, gf)

            df = pd.read_csv(fpath, sep='\t', header=0)
            df['Name'] = df['Name'].str.extract('(?P<ENSID>\w+)')
            ser = pd.Series(df['TPM'].values, index=df['Name'], dtype='float32')

            ser.index.name = sample
            celltype = sample[-2:]
            merger[celltype].append(ser)
        
    with pd.HDFStore(outfile, 'w') as hdf:
        for ct, samples in merger.items():
            mrg = pd.DataFrame(samples, columns=samples[0].index.values,
                               index=[s.index.name for s in samples],
                               dtype='float32')
            mrg.index.name = 'sample'
            mrg = mrg.transpose()

            hdf.put('gene_tpm/{}'.format(ct), mrg, format='fixed')


(1522, 8)
1338


RuntimeError: No active exception to reraise